In [2]:
import pandas as pd
import numpy as np
import random

## Шаг 1.

Прочитайте и проанализируйте данные, выберите турниры, в которых есть данные о составах команд и повопросных результатах (поле mask в results.pkl). 

Для унификации предлагаю:

- взять в тренировочный набор турниры с dateStart из 2019 года; 
- в тестовый — турниры с dateStart из 2020 года.


In [3]:
players = pd.read_pickle('players.pkl')
results = pd.read_pickle('results.pkl')
tournaments = pd.read_pickle('tournaments.pkl')

### players

In [4]:
print(len(players))

204063


In [5]:
for i in sorted(list(players.keys())[100000:100010]):
    print(players[i])

{'id': 112708, 'name': 'Назар', 'patronymic': 'Батырович', 'surname': 'Бекмурадов'}
{'id': 112709, 'name': 'Акджагуль', 'patronymic': '', 'surname': 'Аманова'}
{'id': 112710, 'name': 'Недир', 'patronymic': '', 'surname': 'Кадыров'}
{'id': 112711, 'name': 'Тач', 'patronymic': '', 'surname': 'Нарлыев'}
{'id': 112712, 'name': 'Светлана', 'patronymic': '', 'surname': 'Яковлева'}
{'id': 112713, 'name': 'Михаил', 'patronymic': '', 'surname': 'Дьяконов'}
{'id': 112714, 'name': 'Мяхри', 'patronymic': '', 'surname': 'Баймурадова'}
{'id': 112715, 'name': 'Мурад', 'patronymic': 'Язгельдыевич', 'surname': 'Аннаев'}
{'id': 112716, 'name': 'Владимир', 'patronymic': 'Петрович', 'surname': 'Грамьяк'}
{'id': 112717, 'name': 'Назар', 'patronymic': 'Степанович', 'surname': 'Мищук'}


Мы видим, что табличка содержит просто данные об игроках, в частности их `id` и полные имена.

### tournaments

In [6]:
len(tournaments)

5528

In [7]:
tournaments[4000]

{'id': 4000,
 'name': 'Золотая лестница',
 'dateStart': '2016-09-30T19:30:00+03:00',
 'dateEnd': '2016-10-04T19:29:00+03:00',
 'type': {'id': 3, 'name': 'Синхрон'},
 'season': '/seasons/50',
 'orgcommittee': [{'id': 22053,
   'name': 'Виктор',
   'patronymic': 'Владимирович',
   'surname': 'Мялов'}],
 'synchData': {'dateRequestsAllowedTo': '2016-10-03T23:59:59+03:00',
  'resultFixesTo': '2016-10-14T23:59:59+03:00',
  'resultsRecapsTo': '2016-10-05T23:59:59+03:00',
  'allowAppealCancel': True,
  'allowNarratorErrorAppeal': True,
  'dateArchivedAt': '2016-10-21T23:59:59+03:00',
  'dateDownloadQuestionsFrom': '2016-09-30T00:00:00+03:00',
  'dateDownloadQuestionsTo': '2016-10-04T16:00:00+03:00',
  'hideQuestionsTo': '2016-10-04T23:59:59+03:00',
  'hideResultsTo': '2016-10-04T23:59:59+03:00',
  'allVerdictsDone': None,
  'instantControversial': True},
 'questionQty': {'1': 12, '2': 12, '3': 12}}

Эта табличка -- данные о турнирах.

### results

In [8]:
len(results)

5528

In [9]:
results[4000]

[{'team': {'id': 37542,
   'name': 'Саморазгружающийся полувагон',
   'town': {'id': 201, 'name': 'Москва'}},
  'mask': '001111110101111101011011011011111101',
  'current': {'name': 'Саморазгружающийся полувагон',
   'town': {'id': 201, 'name': 'Москва'}},
  'questionsTotal': 26,
  'synchRequest': {'id': 26407, 'venue': {'id': 3117, 'name': 'Москва'}},
  'position': 1,
  'controversials': [],
  'flags': [],
  'teamMembers': [{'flag': 'Л',
    'usedRating': 7063,
    'rating': 7063,
    'player': {'id': 41351,
     'name': 'Роман',
     'patronymic': 'Александрович',
     'surname': 'Акасов'}},
   {'flag': 'Б',
    'usedRating': 4282,
    'rating': 5139,
    'player': {'id': 44851,
     'name': 'Юлия',
     'patronymic': 'Анатольевна',
     'surname': 'Сычёва'}},
   {'flag': 'Б',
    'usedRating': 2885,
    'rating': 4324,
    'player': {'id': 2698,
     'name': 'Анна',
     'patronymic': 'Игоревна',
     'surname': 'Бахарева'}},
   {'flag': 'К',
    'usedRating': 2164,
    'rating': 43

Эта табличка -- данные о повопросных результатах команд.

In [10]:
for v in results.values():
    assert isinstance(v, list)
    if len(v) > 0:
        if 'mask' in v[0]:
            cond = True
            for item in v:
                if 'mask' not in item:
                    cond = False
        assert cond == True, v

Проверили что если ключ `mask` есть в первом словаре списка команд, то он есть и во всех остальных списках команд. Это правда, значит достаточно проверять только первую команду на наличие такого ключа.

In [11]:
def filter_tournaments(fig, date):
    ans = [k for k, v in fig.items() if v['dateStart'][:4] == str(date)]
    return ans

def filter_results(fig):
    ans = [k for k, v in fig.items() if (len(v) > 0) \
           and ('mask' in v[0].keys()) \
           and  (v[0]['mask'] is not None)]
    return ans

In [12]:
keys_tournaments_train = filter_tournaments(tournaments, 2019)
keys_tournaments_test = filter_tournaments(tournaments, 2020)

keys_results = filter_results(results)

train_keys = list(set(keys_tournaments_train) & set(keys_results))
test_keys = list(set(keys_tournaments_test) & set(keys_results))

In [13]:
print(random.choices(train_keys, k=10))

[5496, 6063, 5156, 5526, 5783, 5479, 5978, 5860, 6135, 6097]


In [14]:
print(random.choices(test_keys, k=10))

[6124, 6285, 5848, 6325, 6247, 5798, 6245, 6247, 5970, 5707]


In [15]:
print(len(train_keys))
print(len(test_keys))

674
173


Можно заметить, что турниров $2020$ году было гораздо меньше, чем в $2019$. Необъяснимый феномен, further research needed.

In [16]:
from operator import itemgetter
train_res = list(itemgetter(*train_keys)(results))
test_res = list(itemgetter(*test_keys)(results))

Также составим словарь где посчитаем общее число вопросов в каждом турнире.

In [17]:
train_tours = list(itemgetter(*train_keys)(tournaments))

In [18]:
def get_table_question_qty(tours, ids):
    defdict = dict()
    for ident, tour in zip(ids, tours):
        qty = sum(tour['questionQty'].values())
        defdict[ident] = qty
    return defdict

In [19]:
qties = get_table_question_qty(train_tours, train_keys)

In [20]:
sum(qties.values())

33311

Всего $33311$ вопросов. Это не очень много, так что мы сможем каждый вопрос из `train` просто пронумеровать.

# Шаг 2.

Постройте baseline-модель на основе линейной или логистической регрессии, которая будет обучать рейтинг-лист игроков. Замечания и подсказки:

- повопросные результаты — это фактически результаты броска монетки, и их предсказание скорее всего имеет отношение к бинарной классификации;
- в разных турнирах вопросы совсем разного уровня сложности, поэтому модель должна это учитывать; скорее всего, модель должна будет явно обучать не только силу каждого игрока, но и сложность каждого вопроса;
- для baseline-модели можно забыть о командах и считать, что повопросные результаты команды просто относятся к каждому из её игроков.


Построим таблицу, где объектами пары игрок-вопрос, где вопрос будет закодирован `one-hot` вектором из всех вопросов. Таргетом мы выберем правильность ответа на соответствующий вопрос.

Также мы в `train` отфильтруем команды, которые пропустили часть вопросов и в целом упростим конфигурацию данных.

In [21]:
def get_team_members(team):
    members = []
    for member in team['teamMembers']:
        ident = member['player']['id']
        members.append(ident)
    return np.array(members, dtype=np.int64)

In [22]:
def get_team_results(members, mask, start_num):
    q_nums = np.array(range(len(mask))) + start_num
    team_results = np.vstack((
        np.tile(members, len(mask)),
        np.repeat(q_nums, len(members)),
        np.repeat(mask, len(members))
        ))
    return team_results
    

In [23]:
def transform_xs(obj):
    if obj == 'X' or obj == '?':
        return 0
    else:
        return int(obj)
    

In [24]:
def create_table(ids, res_list):
    start_num = 1
    all_results = []
    for ident, tournament in zip(ids, res_list):
        total_q = qties[ident]
        for team in tournament:
            if (team['mask'] is None) or (len(team['mask'])) < total_q:
                continue
            else:
                members = get_team_members(team)
                mask = np.array(list(map(transform_xs, team['mask'])), dtype=np.int64) 
                results = np.transpose(get_team_results(members, mask, start_num))
                all_results.append(results)
        start_num += len(mask)
    table_object = np.vstack(all_results)
    return table_object
            

In [25]:
train_rawtable = create_table(train_keys, train_res)

In [26]:
from sklearn.preprocessing import OneHotEncoder
enc = OneHotEncoder(dtype=np.int32)

In [27]:
train_x = enc.fit_transform(train_rawtable[:, :2])
train_y = train_rawtable[:, 2]

/home/markov/anaconda3/lib/python3.7/site-packages/sklearn/preprocessing/_encoders.py:415: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you used a LabelEncoder before this OneHotEncoder to convert the categories to integers, then you can now use the OneHotEncoder directly.
  warnings.warn(msg, FutureWarning)


In [28]:
train_x

<17888755x90739 sparse matrix of type '<class 'numpy.int32'>'
	with 35777510 stored elements in Compressed Sparse Row format>

In [29]:
from sklearn.linear_model import LogisticRegression

In [30]:
baseline_model = LogisticRegression(penalty='l2', solver='liblinear')

In [31]:
baseline_model.fit(train_x, train_y)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='warn', n_jobs=None, penalty='l2',
                   random_state=None, solver='liblinear', tol=0.0001, verbose=0,
                   warm_start=False)

In [32]:
#sanity check
assert len(enc.categories_[0]) + len(enc.categories_[1]) == len(baseline_model.coef_[0])

Построим словарь из `id` в полное имя человека, чтобы было проще оценивать.

In [33]:
coefs_players = baseline_model.coef_[0][:len(enc.categories_[0])]
coefs_qs = baseline_model.coef_[0][len(enc.categories_[0]):]

ids = np.array(enc.categories_[0], dtype = np.dtype(int))

In [34]:
def get_id_to_name(id_list):
    id_to_name = {}
    for ident in id_list:
        full_name = players[ident]['name'] + ' ' + players[ident]['patronymic'] + ' ' + players[ident]['surname']
        id_to_name[ident] = full_name
    return id_to_name

id_to_name = get_id_to_name(enc.categories_[0])

Получим имена топ-20 игроков по нашей `baseline` модели.

In [36]:
top_20_indices = np.argsort(coefs_players)[-20:][::-1]
for num, index in enumerate(top_20_indices):
    print(f'{num+1}.{id_to_name[ids[index]]} с рейтингом {coefs_players[index]}')

1.Максим Михайлович Руссо с рейтингом 4.161364534790012
2.Александра Владимировна Брутер с рейтингом 4.034202057851307
3.Иван Николаевич Семушин с рейтингом 3.9889656651681817
4.Михаил Владимирович Савченков с рейтингом 3.9006253302120606
5.Сергей Леонидович Спешков с рейтингом 3.8218851247659713
6.Артём Сергеевич Сорожкин с рейтингом 3.82069831695976
7.Станислав Григорьевич Мереминский с рейтингом 3.7039529315044937
8.Михаил Ильич Левандовский с рейтингом 3.6494634148697767
9.Ирина Сергеевна Прокофьева с рейтингом 3.60314328524208
10.Сергей Игоревич Николенко с рейтингом 3.5872645404812875
11.Антон Владимирович Саксонов с рейтингом 3.556971597576606
12.Илья Сергеевич Новиков с рейтингом 3.556746054029322
13.Александр Витальевич Либер с рейтингом 3.5392690509823987
14.Игорь Викторович Мокин с рейтингом 3.516631939505424
15.Александр Владимирович Мосягин с рейтингом 3.51631265170102
16.Алексей Владимирович Гилёв с рейтингом 3.5131786053062615
17.Михаил Сергеевич Царёв с рейтингом 3.5043

Запишем наш "рейтинг" каждого игрока в словарь с ключом равным `id` игрока.

In [37]:
def get_id_to_rating(model):
    coefs_players = model.coef_.flatten()[:len(enc.categories_[0])]
    ids = np.array(enc.categories_[0], dtype = np.dtype(int))
    id_to_rating = {k:v for k,v in zip(ids, coefs_players)}
    return id_to_rating

In [38]:
id_to_rating = get_id_to_rating(baseline_model)

# Шаг 3.

Качество рейтинг-системы оценивается качеством предсказаний результатов турниров. Но сами повопросные результаты наши модели предсказывать вряд ли смогут, ведь неизвестно, насколько сложными окажутся вопросы в будущих турнирах; да и не нужны эти предсказания сами по себе. Поэтому:

- предложите способ предсказать результаты нового турнира с известными составами, но неизвестными вопросами, в виде ранжирования команд;

- в качестве метрики качества на тестовом наборе давайте считать ранговые корреляции Спирмена и Кендалла (их можно взять в пакете scipy) между реальным ранжированием в результатах турнира и предсказанным моделью, усреднённые по тестовому множеству турниров.


В качестве способа предсказать результаты нового турнира можно предложить сортировку команд по усредненной силе игроков команды. Игроки с высоким рейтингом будут выше, чем игроки с низким. Новых игроков мы учитывать не будем, то есть будем считать что команда играет без них.

In [39]:
from scipy.stats import spearmanr, kendalltau

In [40]:
def get_team_rating(members, id_to_rating):
    ratings = []
    for member in members:
        rating = id_to_rating.get(member, 0)
        if rating:
            ratings.append(rating)
    if len(ratings) > 0:
        team_rating = np.mean(np.array(ratings))
    else:
        team_rating = 0
    return team_rating

In [41]:
def get_tourn_correlation(tournament, id_to_rating):
    ratings = []
    team_num = len(tournament)
    if team_num > 1:
        true_ranks = np.array(range(team_num))
        for team in tournament:
            members = get_team_members(team)
            rating = get_team_rating(members, id_to_rating)
            ratings.append(rating)
        predicted_ranks = np.argsort(np.array(ratings))[::-1]
        spearm = spearmanr(true_ranks, predicted_ranks)[0]
        k_end_all = kendalltau(true_ranks, predicted_ranks)[0]
    return spearm, k_end_all
    
    
    

In [42]:
def get_test_correlation(test_res, id_to_rating):
    spearms = []
    ks_and_alls = []
    for tournament in test_res:
        if len(tournament) > 1:
            spearm, k_end_all = get_tourn_correlation(tournament, id_to_rating)
            spearms.append(spearm)
            ks_and_alls.append(k_end_all)
        else:
            spearms.append(1)
            ks_and_alls.append(1)
            continue
    if len(spearms) > 0:
        spearm_ans = np.mean(np.array(spearms))
        ks_ans = np.mean(np.array(ks_and_alls))
    else:
        spearm_ans = 1
        ks_ans = 1
    return spearm_ans, ks_ans

In [43]:
final_spearm, final_kendall = get_test_correlation(test_res, id_to_rating=id_to_rating)
print(f'Корелляция Спирмена baseline модели {final_spearm:.2f}')
print(f'Корелляция Кендалла baseline модели {final_kendall:.2f}')

Корелляция Спирмена baseline модели 0.75
Корелляция Кендалла baseline модели 0.59


Для `baseline` модели совсем неплохо!

# Шаг 4.

Теперь главное: ЧГК — это всё-таки командная игра. Поэтому:
        
- предложите способ учитывать то, что на вопрос отвечают сразу несколько игроков; скорее всего, понадобятся скрытые переменные; не стесняйтесь делать упрощающие предположения, но теперь переменные “игрок X ответил на вопрос Y” при условии данных должны стать зависимыми для игроков одной и той же команды;

- разработайте EM-схему для обучения этой модели, реализуйте её в коде;

- обучите несколько итераций, убедитесь, что целевые метрики со временем растут (скорее всего, ненамного, но расти должны), выберите лучшую модель, используя целевые метрики.


Введем скрытые переменные "игрок X ответил на вопрос Y".
На Е-шаге будем оценивать эту вероятность для каждого игрока команды, которая ответила на вопрос Y.
На М-шаге будем обучать логистическую регрессию на метках с Е-шага и по ней оценивать параметры игроков и сложность вопросов.

## Е-шаг

Для всех игроков команды, которые не ответили на вопрос, мы оставляем метку $0$. Для всех игроков, кто правильно ответили на вопрос, мы в качестве метки возьмем вероятность того, что именно этот игрок правильно ответил на вопрос, при условии что ответ был правильным.

$$
p({player}_{ik} = 1| y_i = 1) = \frac{p({player}_{ik} = 1)}{1 - \prod_i^{teamsize}({1 - p({player}_{ik} = 1)})}
$$

Нам понадобятся уникальные id команд.

In [44]:
def get_team_ids(ids, res_list):
    team_ids = []
    for ident, tournament in zip(ids, res_list):
        total_q = qties[ident]
        for team in tournament:
            if (team['mask'] is None) or (len(team['mask'])) < total_q:
                continue
            else:
                num_members = len(team['teamMembers'])
                len_mask = len(team['mask'])
                ids = [team['team']['id']] * num_members * len_mask
                team_ids.extend(ids)
                
    return np.array(team_ids)

team_ids = get_team_ids(train_keys, train_res)

In [45]:
train_df = pd.DataFrame(train_rawtable, columns=['playerId', 'qId', 'target'])
train_df.insert(0, 'teamId', team_ids)

In [46]:
train_df.head(10)

,teamId,playerId,qId,target
0,27254,27469,1,1
1,27254,57286,1,1
2,27254,155103,1,1
3,27254,41104,1,1
4,27254,57288,1,1
5,27254,155102,1,1
6,27254,27469,2,0
7,27254,57286,2,0
8,27254,155103,2,0
9,27254,41104,2,0


In [47]:
def e_step(preds):
    new_df = train_df.copy(deep=True)
    new_df['probas'] =  preds
    new_df['minusprobas'] = 1 - preds
    new_df['prod_minusprobas'] = new_df.groupby(['teamId',
                                                 'qId'])['minusprobas'].transform('prod')
    e_output = (new_df.probas / (1 - new_df.prod_minusprobas)) * new_df.target
    return e_output.values

## М-шаг

На М-шаге нам понадобится обучить логистическую регрессию на "мягких" метках, чтобы получить уточненную аппроксимацию для наших скрытых переменных. Для этого мы применим `inverse_sigmoid` к нашим вероятностям, обучим на полученных значениях линейную регрессию, а к предсказаниям опять применим `sigmoid`-функцию, чтобы отразить полученные значения на отрезок $[0,1]$.

In [48]:
from sklearn.linear_model import LinearRegression, Ridge, Lasso

In [49]:
def inverse_sigmoid(y):
    return np.log(y / (1 - y))

def sigmoid(x):
    return 1 / (1 + np.exp(-x))

In [50]:
def m_step(e_values):
    model = LinearRegression()
    clipped_e = np.clip(e_values, 1e-8, 1 - 1e-8)
    curr_target = inverse_sigmoid(clipped_e)
    model.fit(train_x, curr_target)
    preds = sigmoid(model.predict(train_x))
    return model, preds

In [51]:
def em_process(num_steps):
    baseline_preds = baseline_model.predict_proba(train_x)[:, 1]
    curr_id_to_rating = get_id_to_rating(baseline_model)
    curr_spearm, curr_kendall = get_test_correlation(test_res, curr_id_to_rating)
    print(f'Корелляция Спирмена baseline модели {curr_spearm:.5f}')
    print(f'Корелляция Кендалла baseline модели {curr_kendall:.5f}')
    curr_e = e_step(baseline_preds)
    for i in range(num_steps):
        curr_model, curr_preds = m_step(curr_e)
        curr_e = e_step(curr_preds)
        curr_id_to_rating = get_id_to_rating(curr_model)
        curr_spearm, curr_kendall  = get_test_correlation(test_res, curr_id_to_rating)
        print()
        print(f'Итерация {i+1}')
        print(f'Корелляция Спирмена модели на {i+1} итерации ЕМ-алгоритма {curr_spearm:.5f}')
        print(f'Корелляция Кендалла модели на {i+1} итерации ЕМ-алгоритма {curr_kendall:.5f}')
        print()
    return curr_model

In [52]:
final_model = em_process(4)

Корелляция Спирмена baseline модели 0.75
Корелляция Кендалла baseline модели 0.59

Итерация 1
Корелляция Спирмена модели на 1 итерации ЕМ-алгоритма 0.75315
Корелляция Кендалла модели на 1 итерации ЕМ-алгоритма 0.58815


Итерация 2
Корелляция Спирмена модели на 2 итерации ЕМ-алгоритма 0.75202
Корелляция Кендалла модели на 2 итерации ЕМ-алгоритма 0.58548


Итерация 3
Корелляция Спирмена модели на 3 итерации ЕМ-алгоритма 0.75373
Корелляция Кендалла модели на 3 итерации ЕМ-алгоритма 0.58584


Итерация 4
Корелляция Спирмена модели на 4 итерации ЕМ-алгоритма 0.75701
Корелляция Кендалла модели на 4 итерации ЕМ-алгоритма 0.58784



In [66]:
coefs_players = final_model.coef_[:len(enc.categories_[0])]
coefs_qs = final_model.coef_[len(enc.categories_[0]):]

ids = np.array(enc.categories_[0], dtype = np.dtype(int))

In [67]:
top_20_indices = np.argsort(coefs_players)[-20:][::-1]
for num, index in enumerate(top_20_indices):
    print(f'{num+1}.{id_to_name[ids[index]]} с рейтингом {coefs_players[index]}')

1.Максим Игоревич Пилипенко с рейтингом 19.95153550964062
2.Илья Сергеевич Немец с рейтингом 18.898525199801327
3.Елена Борисовна Бровченко с рейтингом 16.60152033240781
4.Ольга Александровна Козлова с рейтингом 15.37472411675872
5.Майя Александровна Губина с рейтингом 15.27875732409602
6.Дмитрий Сергеевич Кудинов с рейтингом 14.607066705928885
7.Валентина  Подюкова с рейтингом 14.543563363604255
8.Сергей Андреевич Завьялов с рейтингом 14.481579860447747
9.Софья Павловна Молчанова с рейтингом 14.211518101766686
10.Владимир Сергеевич Вакуленко с рейтингом 13.908379151256108
11.Феликс  Фрайман с рейтингом 13.827854347728788
12.Максим Михайлович Руссо с рейтингом 13.6200031665612
13.Анна Сергеевна Печёнкина с рейтингом 13.581737185866917
14.София Романовна Савенко с рейтингом 13.53433784246897
15.София Евгеньевна Лебедева с рейтингом 13.526088103646206
16.Георгий Михайлович Титов с рейтингом 13.456162222148672
17.Светлана Александровна Обогрелова с рейтингом 13.399079731824314
18.Андрей Д

In [136]:
def get_tournament_coefs(keys):
    coef_qs = final_model.coef_[len(enc.categories_[0]):]
    tourn_coefs = []
    for ident in keys:
        total_q = qties[ident]
        curr_coef = coef_qs[:total_q].mean()
        tourn_coefs.append(curr_coef)
        coef_qs = coef_qs[total_q:]
    return tourn_coefs
        
        

In [137]:
tourn_coefs = get_tournament_coefs(train_keys)

In [151]:
tourn_coefs[top_10_difficult_id[0]]

-9.372489698234476

In [152]:
tourn_coefs[top_10_difficult_id[9]]

-5.184689237170314

In [149]:
tourn_coefs[top_10_easy_id[0]]

11.353812286007111

In [150]:
tourn_coefs[top_10_easy_id[9]]

6.9305971875258825

In [148]:
top_10_difficult_id = np.argsort(tourn_coefs)[:10]
top_10_easy_id = np.argsort(tourn_coefs)[-10:][::-1]

In [143]:
print('Топ-10 самых сложных турниров по версии модели.')
for num, index in enumerate(top_10_difficult_id):
    print(f"{num + 1}. {tournaments[train_keys[index]]['name']}")

Топ-10 самых сложных турниров по версии модели.
1. Чемпионат Санкт-Петербурга. Первая лига
2. Угрюмый Ёрш
3. Синхрон высшей лиги Москвы
4. Воображаемый музей
5. Первенство правого полушария
6. Чемпионат Мира. Этап 2. Группа В
7. Чемпионат России
8. Знание – Сила VI
9. Чемпионат Мира. Этап 2. Группа А
10. Ускользающая сова


In [153]:
print('Топ-10 самых простых турниров по версии модели.')
for num, index in enumerate(top_10_easy_id):
    print(f"{num + 1}. {tournaments[train_keys[index]]['name']}")

Топ-10 самых простых турниров по версии модели.
1. One ring - async
2. (а)Синхрон-lite. Лига старта. Эпизод V
3. Синхрон Лиги Разума
4. Школьная лига. I тур.
5. Школьная лига. III тур.
6. Студенческий чемпионат Калининградской области
7. Школьная лига
8. Школьный Синхрон-lite. Выпуск 2.5
9. (а)Синхрон-lite. Лига старта. Эпизод VII
10. (а)Синхрон-lite. Лига старта. Эпизод IX


Наша модель показала неплохой результат! Однако, топ игроков у нее действительно не совпадает с топом `baseline` модели, которая достаточно точно его предсказывала. К сожалению, разобраться в бонусах с этой проблемой мы уже не успеваем :(

![мой любимый мем]()